# Getting to know the ClickHouse-driver Client

This notebook has samples that were included in the [Altinity blog article that introduces the clickhouse-driver client library](https://www.altinity.com/blog/clickhouse-and-python-getting-to-know-the-clickhouse-driver-client).

_WARNING_: If you run the whole notebook it will hang.  You must run the samples one by one as one of them is designed to hang and must be cancelled manually. 

It's easy to load the clickhouse driver. The `Client` class is the main client interface. 

In [1]:
from clickhouse_driver import Client

If you are running against an unencrypted local server setting up a connection is as simple as the following. Instantiating a client does not actually connect to ClickHouse.  It just sets up the data structure used to connect later on when your code does something. 

In [2]:
client = Client('localhost')

Servers with sensitive data should be encrypted with a user/password and encrypted communications.  The following command shows how to connect to a server with self-signed certificate using an explicit database name. 

In [3]:
client = Client('localhost', 
                user='python', 
                password='secret', 
                secure=True, 
                verify=False, 
                database='default',
                compression=True)

The Python driver uses the Client.execute() method to issue select commands.  Results are returned as a list of tuples.  Let's send a very simple query and take apart the results to see values and types. 

*NOTE*: If you get an error about an unknown timezone, ensure your server has the timezone set properly.  

In [4]:
result = client.execute('SELECT now(), version()')
print("RESULT: {0}: {1}".format(type(result), result))
for t in result:
    print(" ROW: {0}: {1}".format(type(t), t))
    for v in t:
        print("  COLUMN: {0}: {1}".format(type(v), v))

RESULT: <class 'list'>: [(datetime.datetime(2019, 2, 24, 8, 53, 18), '19.3.4')]
 ROW: <class 'tuple'>: (datetime.datetime(2019, 2, 24, 8, 53, 18), '19.3.4')
  COLUMN: <class 'datetime.datetime'>: 2019-02-24 08:53:18
  COLUMN: <class 'str'>: 19.3.4


Create the iris table, dropping any previously existing table of the same name. 

In [5]:
r1 = client.execute('DROP TABLE IF EXISTS iris')
print(r1)
r2 = client.execute('CREATE TABLE iris ('
                    'sepal_length Decimal32(2), sepal_width Decimal32(2), '
                    'petal_length Decimal32(2), petal_width Decimal32(2), '
                    'species String) ENGINE = MergeTree '
                    ' PARTITION BY species ORDER BY (species)')
print(r2)

[]
[]


Add some data to the table.  Note that the values are given in a separate array of tuples. 

In [6]:
client.execute(
    'INSERT INTO iris (sepal_length, sepal_width, petal_length, petal_width, species) VALUES',
    [(5.1, 3.7, 1.5, 0.4, 'Iris-setosa'), (4.6, 3.6, 1.0, 0.2, 'Iris-setosa')]
)
print(client.execute("SELECT * FROM iris"))

[(Decimal('5.1'), Decimal('3.7'), Decimal('1.5'), Decimal('0.4'), 'Iris-setosa'), (Decimal('4.6'), Decimal('3.6'), Decimal('1'), Decimal('0.2'), 'Iris-setosa')]


If you try to insert values using a single string INSERT command that works with clickhouse-client the results will be disappointing.  This command will hang.  Once you see it hang, press the STOP button in your Jupyter environment.  

In [7]:
# Don’t do this.
try:
    client.execute(
        "INSERT INTO iris (sepal_length, sepal_width, petal_length, petal_width, species) " 
        "VALUES (5.1, 3.7, 1.5, 0.4, 'Iris-setosa'), (4.6, 3.6, 1.0, 0.2, 'Iris-setosa')"
    )
except:
    print("The command failed as expected")

# This breaks the client connection so we'll need to restore it. 
client = Client('localhost')

The command failed as expected


Here is an example of how to insert CSV.  We read the values line by line using csv.reader() running inside the generator function row_reader().  This results in a tuple for each line.  

In [8]:
import datetime
import csv

# Create a generator to fetch parsed rows.
def row_reader():
    with open('iris.csv') as iris_csv:
        for line in csv.reader(iris_csv):
            yield line

# Use a list comprehension to load values as a list of tuples. 
print(datetime.datetime.now())
client.execute("INSERT INTO iris VALUES", 
               (line for line in row_reader()))
print(datetime.datetime.now())
client.execute("SELECT count(*) FROM iris")

2019-02-24 00:53:36.798925
2019-02-24 00:53:36.819348


[(152,)]

In [9]:
client.execute("TRUNCATE TABLE iris")

import datetime
import csv

# Create a generator to fetch parsed rows. CSV must have variable names in header row.
def row_reader():
    with open('iris_with_names.csv') as iris_csv:
        # Use DictReader to get values as a dictionary with column names.
        for line in csv.DictReader(iris_csv):
            yield {
                'sepal_length': float(line['sepal_length']), 
                'sepal_width': float(line['sepal_width']), 
                'petal_length': float(line['petal_length']), 
                'petal_width': float(line['petal_width']), 
                'species': str(line['species']), 
            }

# Use a generator expression to load values as a list of dictionaries. 
print(datetime.datetime.now())
client.execute("INSERT INTO iris VALUES", (line for line in row_reader()))
print(datetime.datetime.now())
client.execute("SELECT count(*) FROM iris")

2019-02-24 00:53:49.901042
2019-02-24 00:53:49.919261


[(150,)]

The next few queries show examples of select statements. 

In [10]:
result = client.execute('SELECT COUNT(*), species FROM iris '
                        'WHERE petal_length > toDecimal32(3.4, 2) '
                        'GROUP BY species ORDER BY species')
print(result)

[(47, 'Iris-versicolor'), (50, 'Iris-virginica')]


In [11]:
result = client.execute('SELECT COUNT(*), species FROM iris '
                        'WHERE petal_length > toDecimal32(%(max_len)s, 2) '
                        'GROUP BY species ORDER BY species', 
                        {'max_len': 3.4})
print(result)

[(47, 'Iris-versicolor'), (50, 'Iris-virginica')]


Show how to get the column names for results. Note that we also get the column types, which is convenient for conversions. 

In [12]:
result, columns = client.execute('SELECT COUNT(*), species FROM iris '
                                 'WHERE petal_length > toDecimal32(%(max_len)s, 2) '
                                 'GROUP BY species ORDER BY species', 
                                 {'max_len': "a, 2) AND 1=2 AND petal_length < toxDecimal32(3.4"},
                                 with_column_types=True)
print(result)
print(columns)

[(50, 'Iris-setosa'), (50, 'Iris-versicolor'), (50, 'Iris-virginica')]
[('COUNT()', 'UInt64'), ('species', 'String')]


This final example shows how to put a result set into a pandas data frame.  We'll use the column names so that the DataFrame has correct labels.

In [13]:
import pandas
result, columns = client.execute('SELECT * FROM iris WHERE species = %(species)s LIMIT 5', 
                                 {'species': "Iris-setosa"}, with_column_types=True)
pandas.DataFrame(result, columns=[tuple[0] for tuple in columns])

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5,3.6,1.4,0.2,Iris-setosa
